#### Project Background

In today's digital age, online shopping has become the go-to option for many consumers. However, the overwhelming number of choices and the lack of personalized assistance can make the shopping experience daunting. To address this, we have developed **ShopAssist AI, a chatbot that combines the power of large language models and rule-based functions to ensure accurate and reliable information delivery**.


#### Problem Statement

*Given a dataset containing information about laptops (product names, specifications, descriptions, etc.), build a chatbot that parses the dataset and provides accurate laptop recommendations based on user requirements*.


You can load the data and see it here.

In [1]:
# install openai library
!pip install -U -q openai tenacity

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir("/content/drive/MyDrive/GENAI/ShopAssist")
!ls

laptop_data.csv  OpenAI_API_Key.gdoc  OpenAI_API_Key.txt  updated_laptop.csv


In [4]:
# import libraries
import pandas as pd
from IPython.display import display,HTML
# Set the display width to control the output width
pd.set_option('display.width', 100)


In [5]:
# read the data
df=pd.read_csv("laptop_data.csv")

In [6]:
df.head(5)

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [7]:
# Import the libraries
import os, json, ast
import openai
import re
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [8]:
path = '/content/drive/MyDrive/GENAI/ShopAssist/OpenAI_API_Key.txt'
with open(path, 'r') as f:
    openai.api_key = f.read()
os.environ["OpenAI_API_KEY"] = openai.api_key.lstrip().rstrip()

In [9]:
df["Description"][0]

'The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.'

#### Approach:

1. **Conversation and Information Gathering**: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction**: Once the essential information is collected, rule-based functions come into play, extracting top 3 laptops that best matches the user's needs.
3. **Personalized Recommendation**: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the perfect laptop solution.

Let's now look at a brief overview of the major functions that form the chatbot.



- `initialize_conversation()`: This initializes the variable conversation with the system message.
- `get_chat_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant
- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget
- `dictionary_present()`: This function checks if the final understanding of user's profile is returned by the chatbot as a python dictionary or not. If there is a dictionary, it extracts the information as a Python dictionary.
- `compare_laptops_with_user()`: This function compares the user's profile with the different laptops and come back with the top 3 recommendations.
- `initialize_conv_reco()`: Initializes the recommendations conversation

- `initialize_conversation()`

This initializes the variable conversation with the system message. Using prompt engineering and chain of thought reasoning, the function will enable the chatbot to keep asking questions until the user requirements have been captured in a dictionary. It also includes Few Shot Prompting(sample conversation between the user and assistant) to align the model about user and assistant responses at each step.

In [10]:
def initialize_conversation():
  '''
  Returns a list [{'role':"system","content":"system_message"}]
  '''
  delimiter="####"
  example_user_dict= {'GPU intensity': "high",
                        'Display quality':"medium",
                        'Portability': "high",
                        'Multitasking': "high",
                        'Processing speed': "high",
                        'Budget': "150000"}
  example_user_req={'GPU intensity': "_",
                        'Display quality':"_",
                        'Portability': "_",
                        'Multitasking': "_",
                        'Processing speed': "_",
                        'Budget': "_"}
  system_message= f"""

    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this
    {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    All the values in the example dictionary are only representative values.
    {delimiter}
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised:
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'Budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    You are trying to fill the values of all the keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    Answer "Yes" or "No" to indicate if you understand the requirements and have updated the values for the relevant keys. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n

    {delimiter}
    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, I am a programmer."
    Assistant: "Great! As a programmer, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high
    multitasking capability. You would also need a medium display quality. May I know what kind of work do you
    primarily focus on? Are you more involved in coding, testing, or both? Understanding the specific type of programming work
    will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with Coding as I am a Game developer."

    Assistant: "Thank you for providing that information. Working as a Game developer involves working with different programming languages, which will require high GPU. What type of languages do you work on?Understanding your applications will help determine the processing power needed"
    User: "I work using Javascript and Python Script."
    Assistant: "Thank you for the information. Using Python and Javascript will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, I travel alot and carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_dict}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
  conversation = [{'role':"system","content":system_message}]
  return conversation

In [11]:
## Function Description for the Function Calling API
function_descriptions = [
            {
                "name": "compare_laptops_with_user",
                "description": "Get the top 3 laptops from the catalogue, that best matches what the user is asking based on 'GPU intensity','Display quality','Portability','Multitasking','Processing speed' & 'Budget",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "gpu intensity": {
                            "type": "string",
                            "description": "The requirement of the user in GPU capacity classfied as low, medium or high" ,
                        },
                        "display quality": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's Display Quality & capacity classfied as low, medium or high" ,
                        },
                        "portability": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's portability classfied as low, medium or high" ,
                        },
                        "multitasking": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's Multitasking classfied as low, medium or high" ,
                        },
                        "processing speed": {
                            "type": "string",
                            "description": "The requirement of the user for Laptop's Processing speed classfied as low, medium or high" ,
                        },
                        "budget": {
                            "type": "integer",
                            "description": "The maximum budget of the user" ,
                        },

                    },
                    "required": ["GPU intensity","Display quality","Portability","Multitasking","Processing speed","Budget"],
                },
            }
        ]

In [12]:
#This function perform LLM call using the Chat Completions API to get the LLM response.
def get_chat_model_completions(messages):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 300,
        functions=function_descriptions,
        function_call="auto",
    )
    return response.choices[0].message.content


In [13]:
# moderate chatbot conversation
def moderation_check(user_input):
  response=openai.moderations.create(input=user_input)
  moderation_output = response.results[0].flagged
  #print (response.results[0].flagged)
  if response.results[0].flagged==True:
    return 'Flagged'
  else:
    return" Not Flagged"


#### `intent_confirmation_layer()`:

This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not
   - GPU intensity
   - Display quality
   - Portability
   - Multitasking
   - Processing speed
   - Budget

In [14]:
def intent_confirmation_layer(role,response_assistant):
    allowed_values = {'low','medium','high'}
    delimiter = "####"
    content = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 6 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:
    {{
    'GPU intensity': 'values',
    'Display quality':'values',
    'Portability':'values',
    'Multitasking':'values',
    'Processing speed':'values',
    'Budget':'number'
    }}
    The values for the keys should only be from the allowed values: {allowed_values}.
    The 'Budget' key can take only a numerical value.
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    THought 3 - Think carefully before the answering.
    Here is the input: {response_assistant}
    Only output a one-word string - Yes/No.
    """

    confirmation = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
        "role": role,
        "content": content
        }
    ],
    temperature=0,
    max_tokens=256,
    top_p=1,
    )

    return confirmation.choices[0].message.content

In [15]:
#check
debug_response_assistant_1 = f"""
Great, thank you for clarifying your requirements.
Based on your inputs, here is the final profile for the laptop you are looking for:
{{'GPU intensity':'high',
 'Display quality':'high',
 'Portability':'low',
 'Multitasking':'low',
 'Processing speed':'low',
 'Budget':'50000'}}
"""

intent_confirmation_layer('assistant',debug_response_assistant_1)

'Yes'

In [16]:
def dictionary_present(role,input_response):
    delimiter = "####"
    user_req = {'GPU intensity': 'high',
                'Display quality': 'high',
                'Portability': 'medium',
                'Multitasking': 'high',
                'Processing speed': 'high',
                'Budget': '200000'}
    content = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract and return only the python dictionary from the input.
            The output should match the format as {user_req}.
            The output should contain the exact keys and values as present in the input.

            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output 1: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input 2: {{'GPU intensity':     'low', 'Display quality':     'low', 'Portability':    'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90,000'}}
            output 2: {{'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90000'}}

            input 3: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'
            output 3: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000'}}
            {delimiter}
            Here is the input {input_response}

            Don't create any function for extracting the dictionary. Output must be only a python dictionary and not a function for extracting the dictionary.

            """
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
                    {
                    "role": role,
                    "content": content
                    }
                ],
        temperature=0 # this is the degree of randomness of the model's output
    )
    return response.choices[0].message


In [17]:
#extract disctionary from string
def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

In [18]:
#This function compares the user's profile with the different laptops and come back with the top 3 recommendations.
def compare_laptops_with_user(user_req_string):
    print ('\n'+ 'Compare' + user_req_string + '\n')
    laptop_df= pd.read_csv('updated_laptop.csv')
    #This line retrieves the value associated with the key 'budget' from the user_requirements dictionary.
    #If the key is not found, the default value '0' is used.
    user_requirements = extract_dictionary_from_string(user_req_string)
    budget = int(user_requirements.get('budget', '0').replace(',', '').split()[0])


    #The value is then processed to remove commas, split it into a list of strings, and take the first element of the list.
    #Finally, the resulting value is converted to an integer and assigned to the variable budget.


    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()
    #These lines create a copy of the laptop_df DataFrame and assign it to filtered_laptops.
    #They then modify the 'Price' column in filtered_laptops by removing commas and converting the values to integers.
    #Finally, they filter filtered_laptops to include only rows where the 'Price' is less than or equal to the budget.

    mappings = {
        'low': 0,
        'medium': 1,
        'high': 2
    }
    # Create 'Score' column in the DataFrame and initialize to 0
    filtered_laptops['Score'] = 0
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['laptop_feature']
        laptop_values = extract_dictionary_from_string(user_product_match_str)
        score = 0

        for key, user_value in user_requirements.items():
            if key.lower() == 'budget':
                continue  # Skip budget comparison
            laptop_value = laptop_values.get(key, None)
            laptop_mapping = mappings.get(laptop_value.lower(), -1)
            user_mapping = mappings.get(user_value.lower(), -1)
            if laptop_mapping >= user_mapping:
                ### If the laptop value is greater than or equal to the user value the score is incremented by 1
                score += 1

        filtered_laptops.loc[index, 'Score'] = score

    # Sort the laptops by score in descending order and return the top 5 products
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

    return top_laptops.to_json(orient='records')


In [19]:
# Vaidate scores
def recommendation_validation(laptop_recommendation):
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] > 2:
            data1.append(data[i])

    return data1

In [20]:
# initiate conversation
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue: {products}.\
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation


Bringing everything together, we create a `dialogue_mgmt()` function that contains the logic of how the different layers would interact with each other. This will be the function that we'll call to initiate the chatbot

In [21]:
def dialogue_mgmt():
  conversation_bot = []
  conversation = initialize_conversation()
  introduction = get_chat_model_completions(conversation)
  conversation_bot.append({"bot": introduction})
  print(introduction + '\n')
  top_3_laptops = None
  user_input = ''
  while(user_input != "exit"):

      user_input = input("")
      #print('Inside Invite: \n',user_input,'\n')
      prompt = 'Remember your system message and that you are an intelligent laptop assistant. So, you only help with questions around laptop.'
      moderation = moderation_check(user_input)
      if moderation == 'Flagged':
          print("Sorry, this message has been flagged. Please restart your conversation.")

      if top_3_laptops is None:

          conversation.append({"role": "user", "content": user_input + '. ' + prompt})
          conversation_bot.append({'user':user_input})
          #print('top_3_laptops is None\n conversation :',conversation,'\n')
          response_assistant = get_chat_model_completions(conversation)
          print('\n response_assistant : ',response_assistant,'\n')

          moderation = moderation_check(response_assistant)
          if moderation == 'Flagged':
              print("Sorry, this message has been flagged. Please restart your conversation.")
              return
          #print('\n' + response_assistant + '\n')
          conversation_bot.append({'bot':response_assistant})

          # check if input as expected
          confirmation = intent_confirmation_layer('assistant', response_assistant)
          #print(f"confirmation = intent_confirmation_layer('assistant', response_assistant) \n confirmation : {confirmation}")

          if "No" in confirmation:
              conversation.append({"role": "assistant", "content": response_assistant})
              conversation_bot.append({'bot':response_assistant})
          else:
              response = extract_dictionary_from_string(response_assistant)
              print(f"response = extract_dictionary_from_string(response_assistant) \n response : {response}")
              #moderation = moderation_check(response)
              #if moderation == 'Flagged':
              #    print("Sorry, this message has been flagged. Please restart your conversation.")
              #    return

              conversation_bot.append({'bot':"Thank you for providing all the information. Kindly wait, while I fetch the products: \n"})

              print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
              #print('\n' + response + '\n')
              #print(type(response))

              if response.get('function_call'):
                  print(' \n*******\n\nBEGIN Function CALL \n*******\n\n')
                  function_args = json.loads(response.function_call.arguments)
                  print(f"function_args: {function_args}")
                  function_name = response.function_call.name
                  print(f"function_name: {function_name}")


                  if function_name == "compare_laptops_with_user":
                      top_3_laptops = compare_laptops_with_user(function_args)
                      validated_reco = recommendation_validation(top_3_laptops)
                      if len(validated_reco) == 0:
                            conversation_bot.append({'bot':"Sorry, we do not have laptops that match your requirements. Connecting you to a human expert. Please end this conversation."})

                      conversation_reco = initialize_conv_reco(validated_reco)
                      # Step 3: send the info on the function call and function response to GPT
                      conversation_reco.append(response_assistant)
                      conversation_bot.append({"bot": response_assistant.content})
                      conversation_reco.append(
                          {
                            "role": "function",
                            "name": function_name,
                           "content": validated_reco,
                          }
                        )
                  print(' \n*******\n\nEND OF Inside Function CALL \n*******\n\n')
                  recommendation = get_chat_model_completions(conversation_reco)
                  moderation = moderation_check(recommendation)
                  if moderation == 'Flagged':
                      print("Sorry, this message has been flagged. Please restart your conversation.")
                      return

                  conversation_reco.append({"role": "user", "content": "This is my user profile" + response})
                  conversation_reco.append({"role": "assistant", "content": recommendation})
                  conversation_bot.append({'bot':recommendation})

                  print(f"recommendation: {recommendation}")


      else:
          print('Top 3 laptops found\n')
          conversation_reco.append({"role": "user", "content": user_input})
          conversation_bot.append({'user':user_input})

          print('AFter Recommendation - provided, \nConversation_reco:',conversation_reco)

          response_asst_reco = get_chat_model_completions(conversation_reco)
          print('response_asst_reco:',response_asst_reco)

          moderation = moderation_check(response_asst_reco)
          if moderation == 'Flagged':
              print("Sorry, this message has been flagged. Please restart your conversation.")
              return

          #print('\n' + response_asst_reco + '\n')
          #conversation_bot.append({'bot':response_asst_reco})

          conversation.append({"role": "assistant", "content": response_asst_reco})
          conversation_bot.append({'bot':response_asst_reco})

In [ ]:
dialogue_mgmt()

Hi there! I'm here to help you find the best laptop that suits your needs. Could you please share your requirements with me?

Need laptop for coding

 response_assistant :  Great! As a programmer, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a medium display quality. May I know what kind of work do you primarily focus on? Are you more involved in coding, testing, or both? Understanding the specific type of programming work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now. 

coding

 response_assistant :  Thank you for providing that information. Working as a programmer involves working with different programming languages, which will require high GPU. What type of languages do you work on? Understanding your applications will help determine the processing power needed. 

Python

 response_assistant :  Thank you for the information. Usin